<h1>Repeatable splitting</h1>
<br></br>
<p>In this notebook, we will explore the impact of different ways of creating machine learning datasets.

Repeatability is important in machine learning. If you do the same thing now and 5 minutes from now and get different answers, then it makes experimentation difficult. In other words, you will find it difficult to gauge whether a change you made has resulted in an improvement or not.</p>

In [1]:
from google.cloud import bigquery

<h3> Create a simple machine learning model </h3>

The dataset that we will use is <a href="https://bigquery.cloud.google.com/table/bigquery-samples:airline_ontime_data.flights">a BigQuery public dataset</a> of airline arrival data. Click on the link, and look at the column names. Switch to the Details tab to verify that the number of records is 70 million, and then switch to the Preview tab to look at a few rows.
<p>
We want to predict the arrival delay of an airline based on the departure delay. The model that we will use is a zero-bias linear model:
$$ delay_{arrival} = \alpha * delay_{departure} $$
<p>
To train the model is to estimate a good value for $\alpha$. 
<p>
One approach to estimate alpha is to use this formula:
$$ \alpha = \frac{\sum delay_{departure} delay_{arrival} }{  \sum delay_{departure}^2 } $$
Because we'd like to capture the idea that this relationship is different for flights from New York to Los Angeles vs. flights from Austin to Indianapolis (shorter flight, less busy airports), we'd compute a different $alpha$ for each airport-pair.  For simplicity, we'll do this model only for flights between Denver and Los Angeles.

In [2]:
#random splitting
compute_alpha = """
#standardSQL
SELECT 
   SAFE_DIVIDE(SUM(arrival_delay * departure_delay), SUM(departure_delay * departure_delay)) AS alpha
FROM
(
   SELECT RAND() AS splitfield,
   arrival_delay,
   departure_delay
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport = 'DEN' AND arrival_airport = 'LAX'
)
WHERE
  splitfield < 0.8
"""

In [3]:
result = bigquery.Client().query(compute_alpha).to_dataframe()
result

,alpha
0,0.974354


In [4]:
type(result)

pandas.core.frame.DataFrame

In [6]:
alpha = result['alpha'][0]
print(alpha)

0.9743544046975322


<h3> What is wrong with calculating RMSE on the training and test data as follows? </h3>

In [8]:
compute_rmse = """
#standardSQL
SELECT
  dataset,
  SQRT(AVG((arrival_delay - ALPHA * departure_delay)*(arrival_delay - ALPHA * departure_delay))) AS rmse,
  COUNT(arrival_delay) AS num_flights
FROM (
  SELECT
    IF (RAND() < 0.8, 'train', 'eval') AS dataset,
    arrival_delay,
    departure_delay
  FROM
    `bigquery-samples.airline_ontime_data.flights`
  WHERE
    departure_airport = 'DEN'
    AND arrival_airport = 'LAX' )
GROUP BY
  dataset
"""
bigquery.Client().query(compute_rmse.replace('ALPHA', str(alpha))).to_dataframe()

,dataset,rmse,num_flights
0,train,13.077711,63620
1,eval,13.108117,16069


<b>How do we correctly train and evaluate?</b>
<p>Here's the right way to compute the RMSE using the actual training and held-out (evaluation) data. Note how much harder this feels.
Although the calculations are now correct, the experiment is still not repeatable.</p>

<p>Try running it several times; do you get the same answer?</p>

In [9]:
train_and_eval_rand = """
#standardSQL
WITH
  alldata AS (
  SELECT
    IF (RAND() < 0.8,
      'train',
      'eval') AS dataset,
    arrival_delay,
    departure_delay
  FROM
    `bigquery-samples.airline_ontime_data.flights`
  WHERE
    departure_airport = 'DEN'
    AND arrival_airport = 'LAX' ),
  training AS (
  SELECT
    SAFE_DIVIDE( SUM(arrival_delay * departure_delay) , SUM(departure_delay * departure_delay)) AS alpha
  FROM
    alldata
  WHERE
    dataset = 'train' )
SELECT
  MAX(alpha) AS alpha,
  dataset,
  SQRT(AVG((arrival_delay - alpha * departure_delay)*(arrival_delay - alpha * departure_delay))) AS rmse,
  COUNT(arrival_delay) AS num_flights
FROM
  alldata,
  training
GROUP BY
  dataset
"""

In [12]:
bigquery.Client().query(train_and_eval_rand).to_dataframe()

,alpha,dataset,rmse,num_flights
0,0.974597,eval,13.173426,15919
1,0.974597,train,13.061362,63770


<h1>Using HASH of date to split the data</h1>

In [13]:
#standardSQL
compute_alpha = """
#standardSQL
SELECT 
   SAFE_DIVIDE(SUM(arrival_delay * departure_delay), SUM(departure_delay * departure_delay)) AS alpha
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport = 'DEN' AND arrival_airport = 'LAX'
  AND ABS(MOD(FARM_FINGERPRINT(date), 10)) < 8
"""
results = bigquery.Client().query(compute_alpha).to_dataframe()
alpha = results['alpha'][0]
print(alpha)

0.9758039143620403


In [16]:
compute_rmse = """
#standardSQL
SELECT
  IF(ABS(MOD(FARM_FINGERPRINT(date), 10)) < 8, 'train', 'eval') AS dataset,
  SQRT(AVG((arrival_delay - ALPHA * departure_delay)*(arrival_delay - ALPHA * departure_delay))) AS rmse,
  COUNT(arrival_delay) AS num_flights
FROM
    `bigquery-samples.airline_ontime_data.flights`
WHERE
    departure_airport = 'DEN'
    AND arrival_airport = 'LAX'
GROUP BY
  dataset
"""
print(bigquery.Client().query(compute_rmse.replace('ALPHA', str(alpha))).to_dataframe().head())

  dataset       rmse  num_flights
0    eval  12.764685        15671
1   train  13.160712        64018
